In [2]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten,Conv2D,Conv1D,MaxPooling1D,  Dropout
from keras.layers import concatenate
from keras.layers import Bidirectional, GlobalMaxPool1D,BatchNormalization
from keras.models import Model, Sequential
from keras.layers import Convolution1D,GlobalMaxPooling1D
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import to_categorical
import numpy as np
import sys
import os

In [3]:
#df = pd.read_csv("C:/Users/Harvey/Desktop/Yelp_data_set/restuarant_review_balanced.csv"
df = pd.read_csv("/home/ec2-user/Data/restuarant_review_balanced.csv")

In [4]:
df.head()

,sentiment,stars,Processed_Reviews
0,0,1,place go hill clearly cut back staff food qual...
1,0,1,walk around friday afternoon sit table bar wal...
2,0,1,cannot believe things change years pick duck c...
3,1,5,cant really find anything wrong place pastas p...
4,1,4,great lunch today staff helpful assist selecti...


In [5]:
df.Processed_Reviews.apply(lambda x: len(x.split(" "))).mean()

58.18184846606756

In [6]:
train = df.sample(frac = 0.8,random_state = 200)
test = df.drop(train.index)

In [9]:
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train['Processed_Reviews'])
list_tokenized_train = tokenizer.texts_to_sequences(train['Processed_Reviews'])

maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = train['sentiment']
word_index = tokenizer.word_index

In [11]:
embed_size = 128
model = Sequential()
model.add(Embedding(len(word_index)+1, embed_size,input_length = maxlen))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 130, 128)          18109312  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 130, 64)           41216     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1300      
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)      

In [12]:
batch_size = 512
epochs = 3
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 165222 samples, validate on 41306 samples
Epoch 1/3
165222/165222 [==============================] - 762s 5ms/step - loss: 0.1773 - acc: 0.9288 - val_loss: 0.1397 - val_acc: 0.9461
Epoch 2/3
165222/165222 [==============================] - 755s 5ms/step - loss: 0.1206 - acc: 0.9549 - val_loss: 0.1355 - val_acc: 0.9482
Epoch 3/3
165222/165222 [==============================] - 758s 5ms/step - loss: 0.0963 - acc: 0.9647 - val_loss: 0.1315 - val_acc: 0.9510


In [13]:
y_test = test["sentiment"]
list_sentences_test = test['Processed_Reviews']
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
prediction = model.predict(X_te)
y_pred = (prediction > 0.5)
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, confusion_matrix
print('accuracy :{0}'.format(accuracy_score(y_pred, y_test)))
print('F1-score: {0}'.format(f1_score(y_pred, y_test)))
print('Confusion matrix:')
#confusion_matrix(y_pred, y_test)

accuracy :0.9475906414626588
F1-score: 0.9507067910230252
Confusion matrix:


array([[ True],
       [False]])